In [1]:
import numpy as np
import pandas as pd
import plotly.express as pl
from scipy.stats import ttest_ind,t,ttest_1samp

In [8]:
def aa_ttest(mu,mu_0,cov,n_max,size):
    x_0=[]
    n_0=[]
    for i in range(size):
        z_0 = list(np.random.multivariate_normal(mu, cov)[:np.random.randint(1, n_max)])
        n_0 = n_0+[len(z_0)]
        x_0 = x_0+z_0
    x_1=[]
    n_1=[]
    for i in range(size):
        z_1 = list(np.random.multivariate_normal(mu, cov)[:np.random.randint(1, n_max)])
        n_1 = n_1+[len(z_1)]
        x_1 = x_1+z_1    
    return ttest_ind(np.array(x_0),np.array(x_1)).pvalue

In [9]:
def aa_tester(N,alpha,mu,mu_0,cov,n_max,size):
    M=0
    for i in range(N):
        if aa_ttest(mu,mu_0,cov,n_max,size)<alpha:
            M=M+1
    p = M/N
    t_stat = t.ppf(1-alpha/2, N-1)
    se = (p*(1-p)/N)**0.5
    return [p-t_stat*se,p,p+t_stat*se]

In [57]:
aa_tester(1000,0.05,[0]*5,0,[[1,0.5,0.5,0.5,0.5],
                    [0.5,1,0.5,0.5,0.5],
                    [0.5,0.5,1,0.5,0.5],
                    [0.5,0.5,0.5,1,0.5],
                    [0.5,0.5,0.5,0.5,1]],5,500)

[0.14196650771182898, 0.165, 0.18803349228817104]

In [15]:
def aa_ttest_delta(mu,mu_0,cov,n_max,size):
    x=[]
    n=[]
    for i in range(size):
        z = list(np.random.multivariate_normal(mu, cov)[:np.random.randint(1, n_max)])
        n = n+[len(z)]
        x = x+[sum(z)]
        
    z_mean = sum(x)/len(n)
    z_var = np.array(x).std()**2
    y_mean = sum(n)/len(n)
    y_var = np.array(n).std()**2
    cov_0 = np.cov(x,n)[0,1]
    
    var = 1/len(n)*(z_var/y_mean**2+z_mean**2/y_mean**4*y_var-2*z_mean/y_mean**3*cov_0)
    
    
    x_1=[]
    n_1=[]
    for i in range(size):
        z = list(np.random.multivariate_normal(mu, cov)[:np.random.randint(1, n_max)])
        n_1 = n_1+[len(z)]
        x_1 = x_1+[sum(z)]
        
    z_mean_1 = sum(x_1)/len(n_1)
    z_var_1 = np.array(x_1).std()**2
    y_mean_1 = sum(n_1)/len(n_1)
    y_var_1 = np.array(n_1).std()**2
    cov_1 = np.cov(x_1,n_1)[0,1]
    
    var_1 = 1/len(n_1)*(z_var_1/y_mean_1**2+z_mean_1**2/y_mean_1**4*y_var_1-2*z_mean_1/y_mean_1**3*cov_1)
    
    stat = (z_mean/y_mean-z_mean_1/y_mean_1)/(var+var_1)**0.5
    
    
    
    return t.cdf(stat, len(2*n)-2)

In [16]:
def aa_tester_delta(N,alpha,mu,mu_0,cov,n_max,size):
    M=0
    for i in range(N):
        if aa_ttest_delta(mu,mu_0,cov,n_max,size)<alpha:
            M=M+1
    p = M/N
    t_stat = t.ppf(1-alpha/2, N-1)
    se = (p*(1-p)/N)**0.5
    return [p-t_stat*se,p,p+t_stat*se]

In [19]:
aa_tester_delta(1000,0.05,[0]*5,0,[[1,0.5,0.5,0.5,0.5],
                    [0.5,1,0.5,0.5,0.5],
                    [0.5,0.5,1,0.5,0.5],
                    [0.5,0.5,0.5,1,0.5],
                    [0.5,0.5,0.5,0.5,1]],5,500)

[0.03300046902774957, 0.046, 0.05899953097225043]

In [54]:
def aa_ttest_linearization(mu,mu_0,cov,n_max,size):
    x_0=[]
    n_0=[]
    for i in range(size):
        z_0 = list(np.random.multivariate_normal(mu, cov)[:np.random.randint(1, n_max)])
        n_0 = n_0+[len(z_0)]
        x_0.append(sum(z_0))
    x_1=[]
    n_1=[]
    for i in range(size):
        z_1 = list(np.random.multivariate_normal(mu, cov)[:np.random.randint(1, n_max)])
        n_1 = n_1+[len(z_1)]
        x_1.append(sum(z_1))
        
    R_cnt = np.array(x_0).mean()
    
    cnt = np.array(x_0) - R_cnt*np.array(n_0)
    test = np.array(x_1) - R_cnt*np.array(n_1)
    
    return ttest_ind(cnt,test).pvalue


In [55]:
def aa_tester_linearization(N,alpha,mu,mu_0,cov,n_max,size):
    M=0
    for i in range(N):
        if aa_ttest_linearization(mu,mu_0,cov,n_max,size)<alpha:
            M=M+1
    p = M/N
    t_stat = t.ppf(1-alpha/2, N-1)
    se = (p*(1-p)/N)**0.5
    return [p-t_stat*se,p,p+t_stat*se]

In [56]:
aa_tester_linearization(1000,0.05,[0]*5,0,[[1,0.5,0.5,0.5,0.5],
                    [0.5,1,0.5,0.5,0.5],
                    [0.5,0.5,1,0.5,0.5],
                    [0.5,0.5,0.5,1,0.5],
                    [0.5,0.5,0.5,0.5,1]],5,500)

[0.035604378174721625, 0.049, 0.06239562182527838]